In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers xformers
!pip install -q -U accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.3 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [3]:
base_model_id = 'HuggingFaceH4/zephyr-7b-beta'
compute_dtype = getattr(torch, "float16")
dtype = torch.bfloat16 if torch.cuda.get_device_capability()[0] == 8 else compute_dtype
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

print(dtype, DEVICE)

torch.float16 cuda:0


In [4]:
#for better GPU memory management
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
          base_model_id,
          trust_remote_code=True,
          quantization_config=bnb_config,     # can be replaced with "load_in_8bit=True" # for better response but more VRAM req
          torch_dtype = dtype,
          device_map={"": 0})

#tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=True, padding_side='left')

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [43]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=True, padding_side='left', device='cpu')

In [8]:
from transformers import GenerationConfig, TextStreamer

text_generation_config = GenerationConfig(
    temperature = 0.2,
    max_new_tokens = 128,
    repetition_penalty = 1.7,
    num_return_sequences = 1,
    do_sample = True,
    pad_token_id = tokenizer.eos_token_id,
    eos_token_id = tokenizer.eos_token_id,
)

# llama-cpp-python



In [9]:
# CPU llama-cpp-python
!pip install -q llama-cpp-python==0.1.78

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [10]:
%%script true
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 --force-reinstall --upgrade --no-cache-dir --verbose

In [11]:
# For download the models
!pip install huggingface_hub

The prefix 'q5_1' signifies the quantization method we used. I won't delve into too many details, but to determine the best method in each case, I follow the rule that 'q8' yields superior responses at the cost of higher memory usage [slow]. On the other hand, 'q2' may generate subpar responses but requires less RAM [fast].

There are other quantization methods available, and you can read about them in the [model card](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML)

In [12]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

First, we download the model

In [13]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

# Inference with llama-cpp-python

Loading the model

In [14]:
%%script true
# GPU
from llama_cpp import Llama
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096, # Context window
)


For run in CPU
```
# CPU
from llama_cpp import Llama

lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    )
```



In [15]:
# CPU
from llama_cpp import Llama

lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    embedding=True
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [16]:
%%script true
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

We will use this prompt.

In [17]:
prompt = "Write a linear regression in python"
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''

Generating response

If you only use CPU, the response can take a long time. You can reduce the max_tokens to get a faster response.

In [19]:
cpp_model = lcpp_llm.model
cpp_tokenizer = lcpp_llm.tokenizer()

In [97]:
#dir(lcpp_llm)

In [79]:
text = "Hello world, I love Turkiye"
input_ids = cpp_tokenizer.encode(text)#, return_tensors="pt")
input_ids = torch.tensor(input_ids)

In [98]:
input_ids

tensor([    1, 10994,  3186, 29892,   306,  5360,  5383,  1984,  4099])

In [94]:
#query = "How can I bake Pizza at my home?"
#inputs = tokenizer.encode(query, return_tensors='pt').to(model.device)
inputs_embeds = model.model.embed_tokens(input_ids)

##outputs = model.generate(input_ids, inputs_embeds=inputs_embeds)
#print("\ngenerate + inputs_embeds:", tokenizer.decode(outputs[0], skip_special_tokens=True))

#output = model.generate(**inputs, generation_config=generation_config)

In [95]:
inputs_embeds

tensor([[-4.1199e-03,  4.1580e-04, -4.4861e-03,  ..., -4.1771e-04,
         -1.0223e-03, -2.0266e-06],
        [-1.2512e-03, -5.7602e-04,  3.9062e-03,  ...,  1.6632e-03,
         -9.8419e-04, -3.1891e-03],
        [ 5.6152e-03,  2.4719e-03,  2.8801e-04,  ..., -6.5918e-03,
         -3.1891e-03,  3.6774e-03],
        ...,
        [-5.2795e-03, -8.6975e-04, -1.8387e-03,  ..., -5.3101e-03,
         -2.6703e-04, -2.0142e-03],
        [-1.8082e-03, -1.4267e-03,  3.0899e-04,  ...,  4.8256e-04,
         -1.1444e-04, -6.0272e-04],
        [ 1.0681e-03,  1.0204e-04, -4.0283e-03,  ..., -1.8539e-03,
         -2.3193e-03,  6.1035e-04]], device='cuda:0', dtype=torch.float16,
       grad_fn=<EmbeddingBackward0>)

In [111]:
# From inputs_embeds -- exact same output if you also pass `input_ids`. If you don't
# pass `input_ids`, you will get the same generated content but without the prompt
#outputs = model.generate(input_ids, inputs_embeds=inputs_embeds)
outputs = lcpp_llm.generate(input_ids,
                            inputs_embeds=inputs_embeds,
                            )

print("\ngenerate + inputs_embeds:", tokenizer.decode(outputs[0], skip_special_tokens=True))

TypeError: ignored

In [109]:
dir(lcpp_llm)

['__call__',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_candidates',
 '_candidates_data',
 '_candidates_data_id',
 '_candidates_data_p',
 '_convert_text_completion_chunks_to_chat',
 '_convert_text_completion_to_chat',
 '_create_completion',
 '_input_ids',
 '_n_ctx',
 '_n_vocab',
 '_p_tensor_split',
 '_sample',
 '_scores',
 '_token_eos',
 '_token_nl',
 'cache',
 'create_chat_completion',
 'create_completion',
 'create_embedding',
 'ctx',
 'detokenize',
 'embed',
 'eval',
 'eval_logits',
 'eval_tokens',
 'generate',
 'input_ids',
 'last_n_tokens_size',
 'load_state',
 'logits_to_logprobs',
 'longest_token_prefix',
 'lo

In [10]:
%%time
response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=256,
    temperature=0.1,
    top_p=0.95,
    repeat_penalty=1.25,
    top_k=40,
    stop = ['USER:'], # Dynamic stopping when User token is detected.
    echo=True # can be made False to pevent echoing the prompt
)

print(response["choices"][0]["text"])

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: Write a linear regression in python

ASSISTANT:

To write a linear regression in Python, you can use the scikit-learn library. Here is an example of how to do this:
```
from sklearn.linear_model import LinearRegression
import pandas as pd

# Load your dataset into a Pandas DataFrame
df = pd.read_csv('your_data.csv')

# Create X and y arrays from the DataFrame columns
X = df[['feature1', 'feature2']]
y = df['target']

# Initialize the LinearRegression object with the data
reg = LinearRegression().fit(X, y)

# Print the coefficients of the linear regression model
print("Coefficients:", reg.coef_)
```
This code will perform a simple linear regression on the `feature1` and `feature2` columns in your dataset, using the `target` column as the response variable. The `LinearRegression()` constructor takes the data as input and returns an instance of the class with the fitted model. You can then use t

# Inference with langchain




In [11]:
!pip -q install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00


We can use the model that we loaded earlier. However, for illustrative purposes, we will load one from the 'langchain' library. Due to vRAM limitations, before running these cells, you need to delete the previous model from memory.

In [12]:
lcpp_llm.reset()
lcpp_llm.set_cache(None)
lcpp_llm = None
del lcpp_llm


Prompt for the model.

In [13]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from huggingface_hub import hf_hub_download

template = """''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.
USER: {question}
ASSISTANT: """
prompt = PromptTemplate(template=template, input_variables=["question"])

Stream tokens

In [14]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

Load the model

In [15]:
n_gpu_layers = 40  # Change this value based on your model and your GPU VRAM pool.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Loading model,
llm = LlamaCpp(
    model_path=model_path,
    max_tokens=1024,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=True,
    n_ctx=4096, # Context window
    stop = ['USER:'], # Dynamic stopping when such token is detected.
    temperature = 0.4,
)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


Generating response

In [16]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Write a simple linear regression in python"

llm_chain.run(question)

 Of course! Here is an example of how you can write a simple linear regression in Python using scikit-learn library:
```
from sklearn.linear_model import LinearRegression
import pandas as pd

# Load your dataset
df = pd.read_csv('your_data.csv')

# Create a linear regression object and fit the data
reg = LinearRegression()
reg.fit(df[['x1', 'x2']], df['y'])

# Print the coefficients
print(reg.coef_)

# Print the R-squared value
print(reg.score(df[['x1', 'x2']], df['y']))
```
This code will perform a simple linear regression on the data

KeyboardInterrupt: ignored

# Inference with llama.cpp

To use llama.cpp directly, we must clone the repository. In this example, we will use only the CPU.

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp
!git checkout dadbed99e65252d79f81101a392d0d6497b86caa # For compatibility with GGML

Build llama.cpp

We will use the model only with the CPU.

In [ ]:
# CPU
!make

# GPU
#!make LLAMA_CUBLAS=1

In [ ]:
!./main -t 2 -m {model_path} --color -c 128 --temp 0.7 -n 56 -p "USER: Write a linear regression in python\nASSISTANT:"

Details about the different parameters.

<tbody><tr><td class="l"></td><td class="l"> </td><td class="l"> <b>param value</b> </td><td class="l"></td></tr>
  <tr><td class="l">  <code>-h</code> </td><td class="l"> <code>--help</code> </td><td class="l"> </td><td class="l"> Show this help message and exit</td></tr>
  <tr><td class="l">  <code>-i</code> </td><td class="l"> <code>--interactive</code> </td><td class="l"> </td><td class="l"> Run in interactive mode</td></tr>
  <tr><td class="l">  </td><td class="l"> <code>--interactive-first</code> </td><td class="l"> </td><td class="l"> Run in interactive mode and wait for input right away</td></tr>
  <tr><td class="l">  </td><td class="l"> <code>-ins</code>, <code>--instruct</code> </td><td class="l">  </td><td class="l"> Run in instruction mode (use with Alpaca models)</td></tr>
  <tr><td class="l">  <code>-r</code> </td><td class="l"> <code>--reverse-prompt</code> </td><td class="l"> <code>PROMPT</code> </td><td class="l"> Run in interactive mode and poll user input upon seeing <code>PROMPT</code> (can be specified more than once for multiple prompts).</td></tr>
  <tr><td class="l">  </td><td class="l"> <code>--color</code> </td><td class="l">  </td><td class="l"> Colorise output to distinguish prompt and user input from generations</td></tr>
  <tr><td class="l">  <code>-s</code> </td><td class="l"> <code>--seed</code> </td><td class="l"> <code>SEED</code> </td><td class="l"> Seed for random number generator (default: <code>-1</code>, use random seed for &lt;= 0)</td></tr>
  <tr><td class="l">  <code>-t</code> </td><td class="l"> <code>--threads</code> </td><td class="l"> <code>N</code> </td><td class="l"> Number of threads to use during computation (default: 12)</td></tr>
  <tr><td class="l">  <code>-p</code> </td><td class="l"> <code>--prompt</code> </td><td class="l"> <code>PROMPT</code> </td><td class="l"> Prompt to start generation with (default: empty)</td></tr>
  <tr><td class="l">  </td><td class="l"> <code>--random-prompt</code> </td><td class="l"> </td><td class="l"> Start with a randomized prompt.</td></tr>
  <tr><td class="l">  </td><td class="l"> <code>--in-prefix</code> </td><td class="l"> <code>STRING</code> </td><td class="l"> String to prefix user inputs with (default: empty)</td></tr>
  <tr><td class="l">  <code>-f</code> </td><td class="l"> <code>--file</code> </td><td class="l"> <code>FNAME</code> </td><td class="l"> Prompt file to start generation.</td></tr>
  <tr><td class="l">  <code>-n</code> </td><td class="l"> <code>--n_predict</code> </td><td class="l"> <code>N</code> </td><td class="l"> Number of tokens to predict (default: 128, -1 = infinity)</td></tr>
  <tr><td class="l">   </td><td class="l"> <code>--top_k</code> </td><td class="l"> <code>N</code> </td><td class="l"> Top-k sampling (default: 40)</td></tr>
  <tr><td class="l"> </td><td class="l"> <code>--top_p</code> </td><td class="l"> <code>N</code> </td><td class="l"> Top-p sampling (default: 0.9)</td></tr>
  <tr><td class="l"> </td><td class="l"> <code>--repeat_last_n</code> </td><td class="l"> <code>N</code> </td><td class="l"> Last n tokens to consider for penalize (default: 64)</td></tr>
  <tr><td class="l"> </td><td class="l"> <code>--repeat_penalty</code> </td><td class="l"> <code>N</code> </td><td class="l"> Penalize repeat sequence of tokens (default: 1.1)</td></tr>
  <tr><td class="l"> <code>-c</code> </td><td class="l"> <code>--ctx_size</code> </td><td class="l"> <code>N</code> </td><td class="l"> Size of the prompt context (default: <code>512</code>)</td></tr>
  <tr><td class="l"> </td><td class="l"> <code>--ignore-eos</code> </td><td class="l"> </td><td class="l"> Ignore end of stream token and continue generating</td></tr>
  <tr><td class="l"> </td><td class="l"> <code>--memory_f32</code> </td><td class="l"> </td><td class="l"> Use <code>f32</code> instead of <code>f16</code> for memory key+value</td></tr>
  <tr><td class="l"> </td><td class="l"> <code>--temp</code> </td><td class="l"> <code>N</code> </td><td class="l"> Temperature (default: <code>0.8</code>)</td></tr>
  <tr><td class="l">  </td><td class="l"> <code>--n_parts</code> </td><td class="l"> <code>N</code> </td><td class="l"> Number of model parts (default: -1 = determine from dimensions)</td></tr>
  <tr><td class="l">  <code>-b</code> </td><td class="l"> <code>--batch_size</code> </td><td class="l"> <code>N</code> </td><td class="l"> Batch size for prompt processing (default: 8)</td></tr>
  <tr><td class="l">  </td><td class="l"> <code>--perplexity</code> </td><td class="l"> </td><td class="l"> Compute perplexity over the prompt</td></tr>
  <tr><td class="l">  </td><td class="l"> <code>--keep</code> </td><td class="l">   </td><td class="l"> Number of tokens to keep from the initial prompt (default: 0, -1 = all)</td></tr>
  <tr><td class="l">  </td><td class="l"> <code>--mlock</code> </td><td class="l">  </td><td class="l"> Force system to keep model in RAM rather than swapping or compressing</td></tr>
  <tr><td class="l">  </td><td class="l"> <code>--mtest</code> </td><td class="l">  </td><td class="l"> Determine the maximum memory usage needed to do inference for the given <code>n_batch</code> and <code>n_predict</code> parameters (uncomment the <code>"used_mem"</code> line in <code>llama.cpp</code> to see the results)</td></tr>
  <tr><td class="l">  </td><td class="l"> <code>--verbose-prompt</code> </td><td class="l">   </td><td class="l"> Print prompt before generation</td></tr>
  <tr><td class="l">  <code>-m</code> </td><td class="l"> <code>--model</code> </td><td class="l"> <code>FNAME</code> </td><td class="l"> Model path (default: <code>models/llama-7B/ggml-model.bin</code>)</td></tr>

</tbody>




# What is better: GPTQ or GGML?

- GPTQ is a specific format for GPU only.

- GGML is designed for CPU and Apple M series but can also offload some layers on the GPU

- GGMLs like q4_2, q4_3, q5_0, q5_1 and q8_0 have superior inference quality and outperform GPTQ on benchmarks

# References

- [renenyffenegger.ch - LLaMA C++ Library](https://renenyffenegger.ch/notes/development/Artificial-intelligence/language-model/LLM/LLaMA/libs/llama_cpp/)
- [LLaMA C++ Library Documentation](https://llama-cpp-python.readthedocs.io/en/latest/)
- [MacOS Install with Metal GPU - LLaMA C++ Library Documentation](https://llama-cpp-python.readthedocs.io/en/latest/install/macos/)
